# NG-VAT: Sprint-26

## Notebook Configuration

First, some necessary configuration...

In [ ]:
import iris
from iris.experimental.ugrid import PARSE_UGRID_ON_LOAD
import numpy as np
import pyvista as pv

import geovista as gv
from geovista.filters import cast_UnstructuredGrid_to_PolyData as cast

In [ ]:
pv.set_jupyter_backend("pythreejs")

----

## Utility Functions

For convenience sake, create some utility functions to load assorted data that is unstructured and structured...

In [ ]:
def load_fesom(data=False):
    fname = "./tos_Omon_AWI-ESM-1-1-LR_historical_r1i1p1f1_gn_185001-185012.nc"
    cube = iris.load_cube(fname, "tos")[0]

    lons = cube.coord("longitude").bounds
    lats = cube.coord("latitude").bounds
    
    data = cube.data if data else None

    mesh = gv.Transform.from_unstructured(
        lons, lats, lons.shape, data=data, name=cube.name()
    )
    
    if data is None:
        mesh.active_scalars_name = None
    
    return mesh

In [ ]:
def load_lfric(data=False):
    fname = "./qrclim.sst.ugrid.nc"
    with PARSE_UGRID_ON_LOAD.context():
        cube = iris.load_cube(fname)[0]

    data = cube.data if data else None
        
    face_node = cube.mesh.face_node_connectivity
    indices = face_node.indices_by_location()
    lons, lats = cube.mesh.node_coords

    mesh = gv.Transform.from_unstructured(
        lons.points,
        lats.points,
        indices,
        data=data,
        start_index=face_node.start_index,
        name=cube.name(),
    )

    if data is None:
        mesh.active_scalars_name = None
    
    return mesh

In [ ]:
def load_synthetic(data=False):
    M, N = 45, 90
    lats = np.linspace(-90, 90, M + 1)
    lons = np.linspace(-180, 180, N + 1)

    data = np.random.random(M * N) if data else None
        
    mesh = gv.Transform.from_1d(lons, lats, data=data, name="synthetic")
    
    if data is None:
        mesh.active_scalars_name = None
    
    return mesh

## Meridian Extract

In order to support planar projections, we need the ability to determine where to **rip**, or **create a seam** in the mesh at the required **anti-meridian** of the **central longitude** of the chosen projection.

First, create some points-of-interest (POI) to help orientate us on the mesh...

In [ ]:
from geovista.common import to_xyz

poi = to_xyz([0, 0, 0, 90, -180, -90], [90, -90, 0, 0, 0, 0])
poles = to_xyz([0, 0], [90, -90])

Now, load a mesh...

In [ ]:
umesh = load_lfric(data=False)

In [ ]:
umesh

In [ ]:
plotter = gv.GeoPlotter()

plotter.add_mesh(umesh, cmap="balance", show_edges=True)
plotter.add_coastlines()
#plotter.add_base_layer(color="grey")

plotter.show()

In [ ]:
from geovista.core import Slicer, logger

logger.setLevel("INFO")
meridian = 45

slicer = Slicer(umesh, meridian)

bias = "west"
clip = True
mesh_split = slicer.extract(bias=bias, whole_cells=False, clip=clip)
mesh_whole = slicer.extract(bias=bias, split_cells=False, clip=clip)

plotter = gv.GeoPlotter()

plotter.add_base_layer(color="grey")
plotter.add_coastlines()

if mesh_split.n_cells:
    plotter.add_mesh(mesh_split, show_edges=True, color="pink")
if mesh_whole.n_cells:
    plotter.add_mesh(mesh_whole, show_edges=True, color="red")
    
plotter.add_mesh(slicer.slices["exact"], color="green")

plotter.add_points(poi, color="blue", render_points_as_spheres=True, point_size=10)
plotter.show()

----

## Remeshing

Given that we can **extract** the mesh cell faces that **intersect or touch** a specific meridian, we now need the ability to **split cells** that the meridian **cuts through**.

This technique is called **remeshing** and involved triangulating the mesh.

In [ ]:
umesh = load_lfric(data=False)

In [ ]:
umesh

In [ ]:
umesh.active_scalars_name = None

In [ ]:
print(f"{meridian=}")

This remeshing tequnique requires a `PolyData` mesh and a `PolyData` plane to determine the points of intersection for remeshing.

First, let's create a `PolyData` plane for the meridian...

In [ ]:
plane = pv.Plane(center=(0.75, 0, 0), i_resolution=10, j_resolution=10, i_size=1.5, j_size=2.5, direction=(0, 1, 0))
plane.rotate_z(meridian, inplace=True)

Let's see what that looks like...

In [ ]:
plotter = gv.GeoPlotter()

plotter.add_mesh(plane, show_edges=True, opacity=1.0)
#plotter.add_mesh(umesh, cmap="balance", show_edges=False)

plotter.add_coastlines()
plotter.add_axes()
plotter.show()

Ensure that the mesh is a `PolyData` and not an `UnstructuredGrid`...

In [ ]:
mesh_split

In [ ]:
mesh_split = cast(mesh_split)

In [ ]:
mesh_split

Let's remind ourselves what the `mesh_split` mesh looks like...

In [ ]:
plotter = gv.GeoPlotter()

#plotter.add_mesh(plane, show_edges=True, opacity=0.5)
plotter.add_mesh(mesh_split, show_edges=True)

plotter.add_points(poles, color="blue", render_points_as_spheres=True, point_size=10)
plotter.add_coastlines()
plotter.add_axes()
plotter.show()

Time to remesh...

In [ ]:
from geovista.filters import remesh, logger

logger.setLevel("INFO")

triangulated, intersection, remeshed = remesh(mesh_split, plane)

In [ ]:
triangulated.active_scalars_name = None
remeshed.active_scalars_name = None

The intersection mesh contains various useful index information from the **vtkIntersectionPolyDataFilter**...

In [ ]:
intersection

Let's see the triangulated input mesh passed to the **vtkIntersectionPolyDataFilter**...

In [ ]:
plotter = gv.GeoPlotter()

#plotter.add_mesh(plane, show_edges=True, opacity=0.5)
plotter.add_mesh(triangulated, show_edges=True)

plotter.add_points(poles, color="blue", render_points_as_spheres=True, point_size=10)
plotter.add_coastlines()
plotter.add_axes()
plotter.show()

Now let's see the resultant **remeshed** output from the **vtkIntersectionPolyDataFilter**...

In [ ]:
plotter = gv.GeoPlotter()

#plotter.add_mesh(plane, show_edges=True, opacity=0.5)
plotter.add_mesh(remeshed, show_edges=True)

plotter.add_points(poles, color="blue", render_points_as_spheres=True, point_size=10)
plotter.add_coastlines()
plotter.add_axes()
plotter.show()

----

## CRS